In [1]:
import pandas as pd
import math
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import piecewise_regression
import matplotlib.lines as mlines
from typing import List, Dict
from os import path
from config import ROOT_PATH, DATA_PATH

# 1) Read VPA data

In [2]:
# Read catch, and mortality rates (1)
# ---> read weight at age
data_path =path.join(DATA_PATH, 'project', 'vpa', 'weight_age.csv')
w = pd.read_csv(data_path)

# ---> read weight at age SD25
data_path =path.join(DATA_PATH, 'project', 'vpa', 'weight_age_25.csv')
w25 = pd.read_csv(data_path)

# ---> read weight at age
data_path =path.join(DATA_PATH, 'project', 'vpa', 'maturity_rate.csv')
mat = pd.read_csv(data_path)

# ---> read fixed natural mortality matrix
data_path =path.join(DATA_PATH, 'project', 'vpa', 'm_mortality_rate.csv')
m = pd.read_csv(data_path)

# ---> read variable natural mortality matrix
data_path =path.join(DATA_PATH, 'project', 'vpa', 'vm_mortality_rate.csv')
vm = pd.read_csv(data_path)

# ---> read fishing mortality matrix
data_path =path.join(DATA_PATH, 'project', 'vpa', 'f_mortality_rate.csv')
f = pd.read_csv(data_path)

# --> read catch matrix
data_path =path.join(DATA_PATH, 'project', 'vpa', 'catch_rate.csv')
c = pd.read_csv(data_path)

# --> abundance matrix
n = f.copy()
n[n.columns[1:]] = 0

# --> Year pandas series
year_series = c.year


# 2) Supporting Code for Abundance and Fishing Mortality Calculation

In [3]:
class Abundance:
    
    def __init__ (self, c: pd.DataFrame, m: pd.DataFrame, f: pd.DataFrame):
        self.c = c
        self.m = m
        self.f = f
    
    def calculate_abundance_last_cols_rows (self) -> pd.DataFrame:
        n = self.f.copy()
        n[n.columns[1:]] = 0
        
        # Calculate abundance for last column (age 8 for all years)
        ct = self.c.iloc[:, -1]
        zt = self.f.iloc[:, -1] + self.m.iloc[:, -1]
        ft = self.f.iloc[:, -1]
        exp_z = 1 - np.exp(-zt)
        #----> update last column
        n.iloc[:, -1] = round((ct * zt) / (ft * exp_z), 2)

        # Calculate abundance for last row (all ages for the last year)
        ct = self.c.iloc[-1, 1:].reset_index(drop=True)
        zt = (self.f.iloc[-1, 1:] + self.m.iloc[-1, 1:]).reset_index(drop=True)
        ft = self.f.iloc[-1, 1:].reset_index(drop=True)
        exp_z = 1 - np.exp(-zt)
        #----> update last rowex
        n.iloc[-1, 1:] = round((ct * zt) / (ft * exp_z), 2)
        return n
    
    def calculate_abundance_rest_cols_rows (self, n: pd.DataFrame) -> pd.DataFrame:
        for col in range(len(n.columns)-1, 1, -1): # Calculation from age 7 to age 1
            nt1 = n.iloc[1:, col].reset_index(drop=True) # Nt+1,a+1
            e_m = np.exp(self.m.iloc[:-1, col-1])
            em_2 = np.exp(self.m.iloc[:-1, col-1] / 2)
            ct = self.c.iloc[:-1, col-1]
            n.iloc[:-1, col-1] = round((nt1 * e_m) + (ct * em_2), 2)
        return n
    
    def calculate (self) -> pd.DataFrame:
        n = self.calculate_abundance_last_cols_rows()
        n = self.calculate_abundance_rest_cols_rows(n)
        return (n)
    

In [4]:
class FishingMortality:
    
    def __init__(self, n: pd.DataFrame, m: pd.DataFrame, f: pd.DataFrame):
        self.n = n
        self.m = m
        self.f = f.copy()
    
    def calculate_all_but_last_row(self) -> pd.DataFrame:
        for col in range(len(self.n.columns)-1, 1, -1): # from age 8 to age 2 but calucations are from age 7 to 1
            nta1 = self.n.iloc[1:, col].reset_index(drop=True) # Nt+1,a+1
            nta = self.n.iloc[:-1, col-1]
            log_n = np.log(nta / nta1)
            mt = self.m.iloc[:-1, col-1]  # mt = M
            self.f.iloc[:-1, col-1] = round(log_n - mt, 2)
        return self.f
        
    def calculate_average_last_row(self, from_col: int, to_col: int):
        f_average = ((self.f.iloc[:-1, from_col:to_col]).mean(axis=1)).values
        self.f.iloc[:-1, -1] = f_average
        return self.f
    
    def calculate(self, from_col: int, to_col: int):
        self.calculate_all_but_last_row()
        self.calculate_average_last_row(from_col, to_col)
        return self.f
    

# 3) VPA with scalar natural moratlity M

## 3.1) Calculate abundance matrix 

Calculate the abundance matrix by using the initial fishing mortality matrix. 

### 3.1.2) Calculate only the last column and row of N matrix of abundance matrix

To calculate the last row (fish of all ages for the last year) and colums (fish of age 8 for all years), use the catch equation: $ \large N_{t} = \frac{C_{t} * Z_{t}}{F_{t} * \ e^{(1 - Z_{t})}} $

### 3.1.2) Calculate rest of abundance matrix using Pope's equation

The Pope's equation $ \large N_{t, a} =  N_{t+1, a+1}e^{M} + C_{t}e^{\frac{M}{2}} $ is used to calculate the rest of the rows



In [ ]:
abundance = Abundance(c, m, f)
n = abundance.calculate()
n

## 3.2) Recalculate fishing mortality submatrix

### 3.2.1) Recalculate the submatrix that excludes the last column and last row

We exclude the last column from the calculation which corresponds to age 8 and use the formula $ \large log_n (\frac{N_{t,a}}{N_{t+1,a+1}}) - M$

### 3.2.2) Recalculate the last column of the fishing mortality matrix

The last column and for all cells, except the last one the calculation is performed by calculating the average fishing mortality of all ages between 3 and 7. This is because the fishing mortality at early stage of fish life is neglible.

In [ ]:
fishing_mortality = FishingMortality(n, m, f)
from_col, to_col = 3, 8
new_f = fishing_mortality.calculate(from_col, to_col)
new_f

## 3.3) Recalculate the abundance matrix with the new fishing mortality matrix


In [ ]:
abundance = Abundance(c, m, new_f)
n = abundance.calculate()
n

In [ ]:
fishing_mortality = FishingMortality(n, m, f)
from_col, to_col = 3, 8
new_f = fishing_mortality.calculate(from_col, to_col)
new_f

In [ ]:
abundance = Abundance(c, m, new_f)
n = abundance.calculate()
n

In [ ]:
fishing_mortality = FishingMortality(n, m, f)
from_col, to_col = 3, 8
new_f = fishing_mortality.calculate(from_col, to_col)
new_f

In [ ]:
abundance = Abundance(c, m, new_f)
n = abundance.calculate()
n

In [ ]:
fishing_mortality = FishingMortality(n, m, f)
from_col, to_col = 3, 8
new_f = fishing_mortality.calculate(from_col, to_col)
new_f

## 3.4) Calculate SSB, Recruitment and average fishing mortality

To compute the fishing mortality we only take into account ages betgween 3 and 5

In [ ]:
ssb = f.copy()
ssb[ssb.columns[1:]] = 0
ssb.iloc[:, 1:] = w.iloc[:, 1:] * mat.iloc[:, 1:] * n.iloc[:, 1:]
fm_rs_ssb = pd.DataFrame(data={'year': year_series, 
                               'ssb': ssb.iloc[:, 1:].sum(axis=1), 
                               'rs': n.iloc[:, 1],
                               'fm': new_f.iloc[:, 3:6].mean(axis=1).values}) # note that to compute the fishing mortality we only take into account ages from 3 to 6

In [ ]:
fm_rs_ssb

## 3.5) Draw resulting graphs (Average fishing mortality, SSB, Recruitment and yearly class-year fishing mortality)

In [ ]:
_, ax =plt.subplots(nrows=2, ncols=2, figsize=(14,12))
# Plot fishing mortality
sns.scatterplot(data=fm_rs_ssb, 
                x = "year", 
                y="fm", 
                ax=ax[0][0]).set(title = "A) Average Fishing Mortality")
sns.lineplot(data=fm_rs_ssb, x="year", y="fm", ax=ax[0][0])

# Plot Standing Spawning Biomass
sns.scatterplot(data=fm_rs_ssb, 
                x = "year", 
                y="ssb", 
                ax=ax[0][1]).set(title = "B) Standing Spawning Biomass")
sns.lineplot(data=fm_rs_ssb, x="year", y="ssb", ax=ax[0][1])

# Plot Yearly recruitment
sns.scatterplot(data=fm_rs_ssb, 
                x = "year", 
                y="rs", 
                ax=ax[1][0]).set(title = "C) Yearly Recruitment")
sns.lineplot(data=fm_rs_ssb, x="year", y="rs", ax=ax[1][0])

# Plot absolute fishing mortality at eage
f_long = new_f.melt('year', var_name='year_class', value_name='mortality')
sns.pointplot(x="year", y="mortality", hue='year_class', data=f_long, scale=0.5, ax=ax[1][1]).set(title = "D) Fishing mortality at age")
sns.move_legend(ax[1][1], "upper left", bbox_to_anchor=(1, 1), title='Year Class')
ax[1][1].xaxis.set_major_locator(ticker.MaxNLocator(nbins=6))

# Adjust features of all graphs
plt.subplots_adjust(wspace = 0.2, hspace=0.4)
ax[0][0].set_xlabel('')
ax[0][1].set_xlabel('')
ax[1][0].set_xlabel('')
ax[0][0].set_ylabel('Average F')
ax[1][1].set_ylabel('F')
ax[1][1].set_xlabel('Years')

# Plot catch at age
_, ax =plt.subplots(figsize=(6.5,4))
c_long = c.melt('year', var_name='year_class', value_name='catch')
sns.pointplot(x="year", y="catch", hue='year_class', data=c_long, scale=0.5, ax=ax).set(title = "D) Catch at age")
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1), title='Year Class')
ax.xaxis.set_major_locator(ticker.MaxNLocator(nbins=6))
ax.set_xlabel('Years')

# 4) VPA with variable natural mortality M

## 4.1 Calculate the abundance and fihsing mortality matrices

In [6]:
# Calculate the abundance matrix with the new M matrix
abundance = Abundance(c, vm, f)
n_vm = abundance.calculate()
n_vm

,year,1,2,3,4,5,6,7,8
0,1984,50457.19,53320.02,10395.86,6534.46,1456.65,638.41,452.20,325.52
1,1985,34722.01,26139.65,26651.42,5253.59,3221.01,750.13,329.01,246.45
2,1986,20504.78,19802.12,13932.80,13753.03,2778.71,1724.20,431.61,184.76
3,1987,30753.28,12424.20,11473.54,7581.47,7014.34,1500.29,949.28,265.13
4,1988,22146.49,19376.99,7764.28,6399.01,3495.58,3184.63,819.82,514.33
5,1989,43290.17,14343.58,10430.47,4461.96,3273.09,1699.08,1494.35,491.12
6,1990,46754.57,27580.31,9472.80,5603.96,2717.27,1627.22,849.48,701.42
7,1991,58072.30,32723.53,17139.47,6516.85,3258.07,1812.94,912.33,521.41
8,1992,89608.03,42982.49,22428.98,10833.37,4571.50,1967.73,1291.35,556.66
9,1993,98500.99,66847.97,30244.10,14465.73,6834.46,3134.99,1244.13,889.45


In [7]:
# Recalculate the fishing mortality matrix
fishing_mortality = FishingMortality(n_vm, vm, f)
from_col, to_col = 3, 8
new_f_vm = fishing_mortality.calculate(from_col, to_col)
new_f_vm

,year,1,2,3,4,5,6,7,8
0,1984,0.03,0.06,0.05,0.10,0.07,0.08,0.03,0.066
1,1985,0.02,0.09,0.13,0.12,0.11,0.05,0.08,0.098
2,1986,0.03,0.08,0.14,0.21,0.17,0.15,0.05,0.144
3,1987,0.03,0.04,0.15,0.35,0.38,0.20,0.21,0.258
4,1988,0.00,0.19,0.12,0.25,0.31,0.35,0.10,0.226
5,1989,0.06,0.02,0.23,0.12,0.32,0.32,0.39,0.276
6,1990,0.03,0.15,0.04,0.22,0.08,0.26,0.17,0.154
7,1991,0.02,0.10,0.18,0.07,0.22,0.07,0.22,0.152
8,1992,0.02,0.08,0.17,0.19,0.12,0.20,0.11,0.158
9,1993,0.02,0.10,0.13,0.16,0.16,0.10,0.26,0.162


In [8]:
# Recalculate the abundance matrix with new fishing mortality matrix

abundance = Abundance(c, vm, new_f_vm)
n_vm = abundance.calculate()
n_vm

,year,1,2,3,4,5,6,7,8
0,1984,67060.15,68755.59,12430.38,8892.57,2015.63,1444.25,1585.94,1707.21
1,1985,46678.53,34982.25,34872.28,6337.16,4502.29,1059.99,780.20,881.23
2,1986,34506.85,26769.76,19085.81,18591.87,3422.91,2493.60,619.55,458.42
3,1987,35804.81,21175.53,15828.33,10802.11,10069.02,1911.05,1439.87,386.17
4,1988,23061.42,22663.06,13457.10,9231.84,5611.69,5211.87,1095.16,843.18
5,1989,45498.24,14938.75,12568.09,8165.19,5134.39,3103.44,2839.73,673.85
6,1990,48318.09,29075.30,9875.76,7051.25,5249.77,2900.09,1819.52,1630.72
7,1991,59100.83,33847.58,18214.25,6806.55,4309.02,3651.91,1836.62,1225.80
8,1992,96070.15,43759.84,23278.52,11645.67,4790.45,2762.02,2695.18,1262.24
9,1993,108206.07,71781.02,30837.51,15114.25,7454.55,3303.81,1856.57,1971.87


In [9]:
# Recalculate the fishing mortality matrix
fishing_mortality = FishingMortality(n_vm, vm, f)
from_col, to_col = 3, 8
new_f_vm = fishing_mortality.calculate(from_col, to_col)
new_f_vm

,year,1,2,3,4,5,6,7,8
0,1984,0.02,0.05,0.04,0.07,0.05,0.04,0.01,0.042
1,1985,0.02,0.07,0.10,0.10,0.08,0.04,0.03,0.070
2,1986,0.02,0.06,0.10,0.15,0.13,0.10,0.03,0.102
3,1987,0.03,0.02,0.11,0.23,0.25,0.16,0.14,0.178
4,1988,0.00,0.16,0.07,0.17,0.18,0.20,0.08,0.140
5,1989,0.06,0.02,0.19,0.06,0.19,0.16,0.18,0.156
6,1990,0.03,0.14,0.04,0.17,0.04,0.14,0.07,0.092
7,1991,0.02,0.09,0.17,0.07,0.16,0.03,0.11,0.108
8,1992,0.02,0.08,0.16,0.18,0.11,0.14,0.05,0.128
9,1993,0.02,0.10,0.13,0.16,0.15,0.10,0.16,0.140


In [10]:
abundance = Abundance(c, vm, new_f_vm)
n_vm = abundance.calculate()
n_vm

,year,1,2,3,4,5,6,7,8
0,1984,78004.61,85725.64,17501.58,12235.45,2725.97,1964.17,2187.82,2654.09
1,1985,51324.55,40811.18,43910.39,9038.04,6318.65,1453.75,1071.30,1218.22
2,1986,38215.77,29477.22,22482.61,23911.75,5028.64,3584.32,858.38,634.98
3,1987,36457.96,23493.61,17520.50,12925.12,13427.37,2934.91,2135.34,539.99
4,1988,24378.46,23087.94,14965.03,10332.61,7006.61,7440.64,1781.48,1309.37
5,1989,46983.66,15795.50,12844.48,9146.11,5857.65,4029.18,4318.85,1129.33
6,1990,49481.90,30081.01,10455.83,7238.38,5920.58,3394.70,2458.96,2652.40
7,1991,61732.95,34684.27,18937.28,7223.57,4444.90,4139.02,2195.78,1690.13
8,1992,100015.36,45749.15,23910.88,12192.12,5105.63,2864.72,3067.03,1536.42
9,1993,113873.09,74792.71,32356.11,15596.98,7871.70,3546.83,1935.76,2258.59


In [11]:
# Recalculate the fishing mortality matrix
fishing_mortality = FishingMortality(n_vm, vm, f)
from_col, to_col = 3, 8
new_f_vm = fishing_mortality.calculate(from_col, to_col)
new_f_vm

,year,1,2,3,4,5,6,7,8
0,1984,0.02,0.04,0.03,0.05,0.04,0.03,0.01,0.032
1,1985,0.01,0.06,0.08,0.07,0.06,0.03,0.02,0.052
2,1986,0.02,0.05,0.08,0.12,0.09,0.07,0.02,0.076
3,1987,0.03,0.02,0.10,0.19,0.18,0.10,0.09,0.132
4,1988,0.00,0.16,0.06,0.15,0.14,0.13,0.05,0.106
5,1989,0.06,0.02,0.18,0.05,0.17,0.12,0.12,0.128
6,1990,0.03,0.13,0.04,0.17,0.04,0.12,0.05,0.084
7,1991,0.02,0.09,0.16,0.07,0.16,0.03,0.09,0.102
8,1992,0.02,0.08,0.16,0.17,0.10,0.13,0.05,0.122
9,1993,0.02,0.09,0.12,0.15,0.14,0.09,0.16,0.132


In [12]:
abundance = Abundance(c, vm, new_f_vm)
n_vm = abundance.calculate()
n_vm

,year,1,2,3,4,5,6,7,8
0,1984,80229.53,89741.93,20057.12,15056.32,3525.51,2574.24,2917.16,3467.89
1,1985,52528.31,41996.15,46049.43,10399.10,7851.37,1896.96,1412.88,1626.58
2,1986,39871.56,30178.71,23173.15,25170.80,5837.82,4504.71,1127.20,842.16
3,1987,37319.27,24528.48,17958.93,13356.71,14222.19,3450.86,2722.21,713.12
4,1988,25334.11,23648.23,15638.22,10617.81,7290.18,7968.12,2127.33,1702.76
5,1989,48537.75,16417.16,13208.95,9584.03,6045.04,4217.37,4668.91,1358.85
6,1990,50478.11,31133.22,10876.73,7485.15,6220.06,3522.85,2588.95,2894.20
7,1991,63325.28,35400.47,19693.74,7526.17,4624.09,4356.49,2288.84,1784.52
8,1992,102075.67,46952.61,24452.17,12763.84,5334.33,3000.15,3233.04,1607.46
9,1993,116920.26,76365.51,33274.81,16010.19,8308.14,3723.17,2040.18,2386.59


In [13]:
# Recalculate the fishing mortality matrix
fishing_mortality = FishingMortality(n_vm, vm, f)
from_col, to_col = 3, 8
new_f_vm = fishing_mortality.calculate(from_col, to_col)
new_f_vm

,year,1,2,3,4,5,6,7,8
0,1984,0.02,0.04,0.03,0.04,0.03,0.02,0.00,0.024
1,1985,0.01,0.05,0.07,0.06,0.05,0.02,0.02,0.044
2,1986,0.02,0.05,0.08,0.11,0.08,0.05,0.02,0.068
3,1987,0.03,0.02,0.10,0.19,0.17,0.08,0.07,0.122
4,1988,0.00,0.15,0.06,0.14,0.14,0.12,0.04,0.100
5,1989,0.05,0.02,0.18,0.05,0.16,0.12,0.11,0.124
6,1990,0.02,0.13,0.04,0.16,0.04,0.11,0.05,0.080
7,1991,0.02,0.09,0.15,0.06,0.15,0.03,0.08,0.094
8,1992,0.02,0.07,0.15,0.16,0.10,0.13,0.04,0.116
9,1993,0.02,0.09,0.12,0.15,0.13,0.09,0.15,0.128


In [17]:
abundance = Abundance(c, vm, new_f_vm)
n_vm_1 = abundance.calculate()
n_vm_1

,year,1,2,3,4,5,6,7,8
0,1984,83637.99,92051.44,21530.48,16004.77,3952.46,2936.97,3592.55,4607.25
1,1985,54330.82,43811.47,47279.46,11183.80,8366.71,2133.63,1615.97,2004.73
2,1986,41741.12,31229.12,24231.02,25894.80,6304.34,4814.17,1270.75,965.34
3,1987,38012.42,25696.96,18615.44,14017.88,14679.24,3748.33,2919.53,805.57
4,1988,26328.96,24099.13,16398.33,11044.88,7724.60,8271.44,2326.73,1835.03
5,1989,49656.18,17064.32,13502.27,10078.49,6325.64,4505.67,4870.21,1491.18
6,1990,50977.86,31890.46,11314.89,7683.74,6558.20,3714.74,2788.09,3033.24
7,1991,64198.81,35759.75,20238.14,7841.17,4768.30,4602.03,2428.18,1929.13
8,1992,102851.27,47612.81,24723.71,13175.29,5572.40,3109.14,3420.48,1713.83
9,1993,118460.50,76957.59,33778.79,16217.48,8622.23,3906.73,2124.22,2531.12


In [18]:
# Recalculate the fishing mortality matrix
fishing_mortality = FishingMortality(n_vm_1, vm, f)
from_col, to_col = 3, 8
new_f_vm = fishing_mortality.calculate(from_col, to_col)
new_f_vm

,year,1,2,3,4,5,6,7,8
0,1984,0.02,0.04,0.03,0.04,0.03,0.02,0.00,0.024
1,1985,0.01,0.05,0.07,0.05,0.04,0.02,0.02,0.040
2,1986,0.02,0.05,0.08,0.11,0.07,0.05,0.02,0.066
3,1987,0.03,0.02,0.09,0.18,0.16,0.08,0.06,0.114
4,1988,0.00,0.15,0.06,0.14,0.13,0.12,0.03,0.096
5,1989,0.05,0.02,0.17,0.05,0.15,0.11,0.10,0.116
6,1990,0.02,0.12,0.04,0.16,0.03,0.11,0.05,0.078
7,1991,0.02,0.09,0.15,0.06,0.15,0.03,0.08,0.094
8,1992,0.02,0.07,0.15,0.15,0.10,0.12,0.04,0.112
9,1993,0.02,0.09,0.12,0.15,0.13,0.08,0.14,0.124


In [21]:
abundance = Abundance(c, vm, new_f_vm)
n_vm_2 = abundance.calculate()
n_vm_2

,year,1,2,3,4,5,6,7,8
0,1984,83637.99,93295.14,21530.48,16266.62,4014.31,2936.97,3768.16,4607.25
1,1985,54821.88,43811.47,47941.84,11183.80,8508.99,2167.91,1615.97,2103.05
2,1986,41741.12,31515.28,24231.02,26284.68,6304.34,4899.61,1291.54,965.34
3,1987,38159.89,25696.96,18794.29,14017.88,14925.36,3748.33,2974.01,818.96
4,1988,26328.96,24195.06,16398.33,11161.22,7724.60,8434.78,2326.73,1871.55
5,1989,49790.96,17064.32,13564.67,10078.49,6402.08,4505.67,4978.61,1491.18
6,1990,50977.86,31981.71,11314.89,7725.99,6558.20,3767.01,2788.09,3108.12
7,1991,64198.81,35759.75,20303.74,7841.17,4798.98,4602.03,2466.13,1929.13
8,1992,103121.09,47612.81,24723.71,13224.87,5572.40,3132.33,3420.48,1742.80
9,1993,118460.50,77163.56,33778.79,16217.48,8660.08,3906.73,2142.10,2531.12


## 4.2 Calculate SSB, Recruitment and average fishing mortality


In [ ]:
ssb_vm = f.copy()
ssb_vm[ssb.columns[1:]] = 0
ssb_vm.iloc[:, 1:] = w.iloc[:, 1:] * mat.iloc[:, 1:] * n_vm.iloc[:, 1:]
fm_rs_ssb_vm = pd.DataFrame(data={'year': year_series, 
                               'ssb': ssb_vm.iloc[:, 1:].sum(axis=1), 
                               'rs': n_vm.iloc[:, 1],
                               'fm': new_f_vm.iloc[:, 3:6].mean(axis=1).values}) # note that to compute the fishing mortality we only take into account ages from 3 to 6

## 4.3 Draw resulting graphs (Average fishing mortality, SSB, Recruitment and yearly class-year fishing mortality)

In [ ]:
_, ax =plt.subplots(nrows=2, ncols=2, figsize=(14,12))
# Plot fishing mortality
sns.scatterplot(data=fm_rs_ssb_vm, 
                x = "year", 
                y="fm", 
                ax=ax[0][0]).set(title = "A) Average Fishing Mortality")
sns.lineplot(data=fm_rs_ssb_vm, x="year", y="fm", ax=ax[0][0])

# Plot Standing Spawning Biomass
sns.scatterplot(data=fm_rs_ssb_vm, 
                x = "year", 
                y="ssb", 
                ax=ax[0][1]).set(title = "B) Standing Spawning Biomass")
sns.lineplot(data=fm_rs_ssb_vm, x="year", y="ssb", ax=ax[0][1])

# Plot Yearly recruitment
sns.scatterplot(data=fm_rs_ssb_vm, 
                x = "year", 
                y="rs", 
                ax=ax[1][0]).set(title = "C) Yearly Recruitment")
sns.lineplot(data=fm_rs_ssb_vm, x="year", y="rs", ax=ax[1][0])

# Plot absolute fishing mortality at eage
f_long_vm = new_f_vm.melt('year', var_name='year_class', value_name='mortality')
sns.pointplot(x="year", y="mortality", hue='year_class', data=f_long_vm, scale=0.5, ax=ax[1][1]).set(title = "D) Fishing mortality at age")
sns.move_legend(ax[1][1], "upper left", bbox_to_anchor=(1, 1), title='Year Class')
ax[1][1].xaxis.set_major_locator(ticker.MaxNLocator(nbins=6))

# Adjust features of all graphs
plt.subplots_adjust(wspace = 0.2, hspace=0.4)
ax[0][0].set_xlabel('')
ax[0][1].set_xlabel('')
ax[1][0].set_xlabel('')
ax[0][0].set_ylabel('Average F')
ax[1][1].set_ylabel('F')
ax[1][1].set_xlabel('Years')

# Plot catch at age
_, ax =plt.subplots(figsize=(6.5,4))
c_long = c.melt('year', var_name='year_class', value_name='catch')
sns.pointplot(x="year", y="catch", hue='year_class', data=c_long, scale=0.5, ax=ax).set(title = "D) Catch at age")
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1), title='Year Class')
ax.xaxis.set_major_locator(ticker.MaxNLocator(nbins=6))
ax.set_xlabel('Years')

# 5) Comparative graphs for Fixed and Variable Mortality

In [ ]:
_, ax =plt.subplots(nrows=2, ncols=2, figsize=(14,8))

# 1) Natural mortality comparison when M fixed vs variable
mf_vs_mv = pd.DataFrame(data={'year': year_series, 'm': m.iloc[:, 1:].mean(axis=1), 'm_type': 'Fixed M'})
mvar = pd.DataFrame(data={'year': year_series, 'm': vm.iloc[:, 1:].mean(axis=1), 'm_type': 'Variable M'})
mf_vs_mv = pd.concat([mf_vs_mv, mvar], axis=0, keys=['m', 'm']).reset_index()

sns.scatterplot(data=mf_vs_mv, 
                x = "year", 
                y="m",
                hue='m_type',
                palette=['r', 'g'], 
                ax = ax[0][0]).set(title = "A) Natural Mortality")
sns.lineplot(data=mf_vs_mv, x='year', y='m', hue='m_type', palette=['r', 'g'], ax = ax[0][0])
ax[0][0].legend(title= "M (variable vs fixed)")

# 2) Fishing mortality comparison when M fixed vs variable
ff_vs_fv = pd.DataFrame(data={'year': year_series, 'fm': fm_rs_ssb.fm, 'm_type': 'Fixed M'})
fv = pd.DataFrame(data={'year': year_series, 'fm': fm_rs_ssb_vm.fm, 'm_type': 'Variable M'})
ff_vs_fv = pd.concat([ff_vs_fv, fv], axis=0, keys=['fm', 'fm']).reset_index()

sns.scatterplot(data=ff_vs_fv, 
                x = "year", 
                y="fm",
                hue='m_type',
                palette=['r', 'g'], 
                ax = ax[0][1]).set(title = "B) Fishing Mortality")
sns.lineplot(data=ff_vs_fv, x='year', y='fm', hue='m_type', palette=['r', 'g'], ax = ax[0][1])
ax[0][1].legend(title= "F (variable vs fixed)")


# 3) SSB comparison when M fixed vs variable
ssbf_vs_ssbv = pd.DataFrame(data={'year': year_series, 'ssb': fm_rs_ssb.ssb, 'm_type': 'Fixed M'})
ssbv = pd.DataFrame(data={'year': year_series, 'ssb': fm_rs_ssb_vm.ssb, 'm_type': 'Variable M'})
ssbf_vs_ssbv = pd.concat([ssbf_vs_ssbv, ssbv], axis=0, keys=['ssb', 'ssb']).reset_index()

sns.scatterplot(data=ssbf_vs_ssbv,
                x = "year",
                y="ssb",
                hue='m_type',
                palette=['r', 'g'], 
                ax = ax[1][0]).set(title = "C) Spawning Stock Biomass")
sns.lineplot(data=ssbf_vs_ssbv, x='year', y='ssb', hue='m_type', palette=['r', 'g'], ax = ax[1][0])
ax[1][0].legend(title= "SSB (variable vs fixed)")

# 4) Year Recruitment comparison when M fixed vs variable
rsf_vs_rsbv = pd.DataFrame(data={'year': year_series, 'rs': fm_rs_ssb.rs, 'm_type': 'Fixed M'})
rsv = pd.DataFrame(data={'year': year_series, 'rs': fm_rs_ssb_vm.rs, 'm_type': 'Variable M'})
rsf_vs_rsbv = pd.concat([rsf_vs_rsbv, rsv], axis=0, keys=['rs', 'rs']).reset_index()

sns.scatterplot(data=rsf_vs_rsbv,
                x = "year",
                y="rs",
                hue='m_type',
                palette=['r', 'g'], 
                ax = ax[1][1]).set(title = "D) Year Recruitment")
sns.lineplot(data=rsf_vs_rsbv, x='year', y='rs', hue='m_type', palette=['r', 'g'], ax = ax[1][1])
ax[1][1].legend(title= "SSB (variable vs fixed)")
ax[1][1].legend(title= "Recruitment (variable vs fixed)")

# Remove dots legend
for i in range(2):
    for j in range(2):
        handles, labels = ax[i][j].get_legend_handles_labels()
        ax[i][j].legend(handles[2:], labels)

plt.subplots_adjust(wspace = 0.2, hspace=0.4)



# 6) VPA with WAA SD25

## 6.1 Compare the two matrices of weights and SSBs

In [ ]:
w_av = pd.DataFrame(data={'year':year_series , 'av_weight':w.iloc[:, 1:].mean(axis=1), 'w_type': 'Global WAA'})
w_25_av = pd.DataFrame(data={'year':year_series , 'av_weight':w25.iloc[:, 1:].mean(axis=1), 'w_type': 'WAA SD25'})
ws = pd.DataFrame(data={'w_av': w_av.iloc[:, 1:].av_weight, 'w_25_av': w_25_av.iloc[:, 1:].av_weight})
ws.describe()

In [ ]:
# Build the weight matrices to be compared
w_vs_w25 = pd.concat([w_av, w_25_av], axis=0, keys=['av_weight', 'av_weight']).reset_index()

# Build the ssb matrices to be compared.

ssb_global = pd.DataFrame(data={'year': year_series, 'ssb': fm_rs_ssb.ssb, 'ssb_type': 'Global WAA'})

ssb_25 = f.copy()
ssb_25[ssb_25.columns[1:]] = 0
ssb_25.iloc[:, 1:] = w25.iloc[:, 1:] * mat.iloc[:, 1:] * n.iloc[:, 1:]
ssb_25 = pd.DataFrame(data={'year': year_series, 'ssb': ssb_25.iloc[:, 1:].sum(axis=1), 'ssb_type': 'WAA SD25'})
ssb_vs_ssb25 = pd.concat([ssb_global, ssb_25], axis=0, keys=['ssb', 'ssb']).reset_index()
ssb_vs_ssb25 

In [ ]:
_, ax =plt.subplots(nrows=1, ncols=2, figsize=(12,6))

# plot the global weight vs weight 25
s_plot =sns.scatterplot(data=w_vs_w25, 
                x = "year", 
                y="av_weight",
                hue='w_type',
                palette=['r', 'g'], 
                ax=ax[0]).set(title = "Gloval Weight vs Weight @ SD25 ")
l_plot = sns.lineplot(data=w_vs_w25, x='year', y='av_weight', hue='w_type', palette=['r', 'g'], ax=ax[0])
ax[0].set_xlabel('Years')
ax[0].set_ylabel('Weight')

# plot the global ssb vs ssb 25
s_plot =sns.scatterplot(data=ssb_vs_ssb25, 
                x = "year", 
                y="ssb",
                hue='ssb_type',
                palette=['r', 'g'], 
                ax=ax[1]).set(title = "Global SSB vs SSB @ SD25 ")
l_plot = sns.lineplot(data=ssb_vs_ssb25, x='year', y='ssb', hue='ssb_type', palette=['r', 'g'], ax=ax[1])
ax[1].set_xlabel('Years')
ax[1].set_ylabel('SSB')

# Remove dots legend
for i in range(2):
    handles, labels = ax[1].get_legend_handles_labels()
    ax[i].legend(handles[2:], labels)
    

# 6) S-R relationships

* S-R with fixed mortality rate
* S-R with variable mortality rate
* S-R with WAA SD25

## 6.1) Describe the different SSB (fixed M, variable M and different WAA)

In [ ]:
# Describe different ssbs
ssb_m = fm_rs_ssb.ssb
ssb_vm = fm_rs_ssb_vm.ssb
ssb_wa_25 = ssb_25.ssb
ssbs = pd.DataFrame(data={'ssbm': ssb_m, 'ssbvm': ssb_vm, 'ssb_waa_25': ssb_wa_25})
ssbs.describe()


## 6.2) Describe the different Recruitments (fixed M and variable M)


In [ ]:
# Describe different recruitments 
rs_m = fm_rs_ssb.rs
rs_vm = fm_rs_ssb_vm.rs
rss = pd.DataFrame(data={'rsm': rs_m, 'rsvm': rs_vm,})
rss.describe()

## 6.3)S-R relationships with fixed M

In [ ]:
s_r_fm = pd.DataFrame(data={'ssb': ssb_m, 'rs': rs_m, 's_r_type': 'Fixed M'})
s_r_vm = pd.DataFrame(data={'ssb': ssb_vm, 'rs': rs_vm, 's_r_type': 'Variable M'})
s_r_fm_waa_25 = pd.DataFrame(data={'ssb': ssb_wa_25, 'rs': rs_m, 's_r_type': 'Fixed M/WAA SD25'})
s_r = pd.concat([s_r_fm, s_r_vm, s_r_fm_waa_25], axis=0, keys=['rs', 'rs', 'rs']).reset_index()

In [ ]:
_, ax =plt.subplots(nrows=2, ncols=2, figsize=(14,10))
s_r_fm_plt =sns.scatterplot(data=s_r_fm, 
                x = "ssb", 
                y="rs", 
                ax=ax[0][0]).set(title = "A) S-R for fixed M")

s_r_vm_plt = sns.scatterplot(data=s_r_vm, 
                x = "ssb", 
                y="rs",  
                hue=[1] * len(s_r_vm),
                palette=["r"],
                ax=ax[0][1]).set(title = "A) S-R for Variable M")

s_r_25_plt = sns.scatterplot(data=s_r_fm_waa_25, 
                x = "ssb", 
                y="rs",  
                hue=[1] * len(s_r_fm_waa_25),
                palette=["g"],
                ax=ax[1][0]).set(title = "A) S-R for WAA @ SD25")

s_r_plt = sns.scatterplot(data=s_r, 
                x = "ssb", 
                y="rs",  
                hue='s_r_type',
                palette=['b', 'r', 'g'],
                ax=ax[1][1]).set(title = "A) ALL overlapped S-R")

plt.subplots_adjust(wspace = 0.2, hspace=0.2)

# 7) Blim calculation

## 7.1) Calculate BLIM following the theory that is Spasmodic (Fixed mortality)

In [ ]:
pw_fit = piecewise_regression.Fit(ssb_m.values, rs_m.values, n_breakpoints=1)


In [ ]:
ssb_rs = pd.DataFrame(data={'year': year_series, 'ssb': np.log(fm_rs_ssb.ssb), 'rs': np.log(fm_rs_ssb.rs)})
ssb_rs_long = pd.melt(ssb_rs, id_vars=['year'], value_vars=['ssb', 'rs'], var_name='data_type', value_name='values')

In [ ]:
def build_twin_lineplot_legend(data: List[Dict]):
    return ([mlines.Line2D([], [], color=x['color'], label=x['label']) for x in data])


In [ ]:
_, ax =plt.subplots(nrows=2, ncols=2, figsize=(14,14))

# # Plot segementation regrestion
pw_fit.plot()
ax[1][1].set_title("D) S-R relationship and segmented regression")
ax[1][1].set_xlabel("SSB")
ax[1][1].set_ylabel("Recruitment")

# Plot Year class recruitment and fishing mortality
ax_00 = ax[0][0].twinx()
sns.scatterplot(data=fm_rs_ssb,
                x = "year",
                y="rs", 
                color='g',
                ax = ax[0][0])
sns.lineplot(data=fm_rs_ssb, 
             x='year', 
             y='rs', 
             color='g',
             ax = ax[0][0])
sns.scatterplot(data=fm_rs_ssb,
                x = "year",
                y="fm",
                color='r',
                ax = ax_00)
sns.lineplot(data=fm_rs_ssb, 
             x='year', 
             y='fm', 
             color='r', 
             ax = ax_00)
ax_00.set_ylabel("Fishing Mortality")
ax[0][0].set(xticks=year_series)
ax[0][0].set_xticklabels(ax[0][0].get_xticks(), rotation = 90)
ax[0][0].set_title("A) Year Recruitment and Fishing Mortality")
ax[0][0].set_xlabel("Years")
ax[0][0].set_ylabel("Recruitment")
handles = build_twin_lineplot_legend ([{'color': 'g', 'label': 'Recruitment'}, 
                             {'color': 'r', 'label': 'Fishing Mortality'}])
ax[0][0].legend(handles = handles)
ax[0][0].grid()


# Plot Year class recruitment and SSB
ax_01 =ax[0][1].twinx()

sns.scatterplot(data=fm_rs_ssb,
                x = "year",
                y="rs", 
                 color='g',
                ax = ax[0][1])
sns.lineplot(data=fm_rs_ssb, 
             x='year', 
             y='rs', 
             color='g',
             ax = ax[0][1])
sns.scatterplot(data=fm_rs_ssb,
                x = "year",
                y="ssb",
                color='r',
                ax = ax_01)
sns.lineplot(data=fm_rs_ssb, 
             x='year', 
             y='ssb', 
             color='r', 
             ax = ax_01)
sns.lineplot(x=year_series, 
             y=[fm_rs_ssb.ssb.mean()] * len(year_series), 
             color='r',
             linestyle='--',
             ax = ax_01)
ax_01.set_ylabel("SSB")
ax[0][1].set(xticks=year_series)
ax[0][1].set_xticklabels(ax[0][0].get_xticks(), rotation = 90)
ax[0][1].set_title("B) Year Recruitment and SSB")
ax[0][1].set_xlabel("Years")
ax[0][1].set_ylabel("Recruitment")
handles = build_twin_lineplot_legend ([{'color': 'g', 'label': 'Recruitment'}, 
                             {'color': 'r', 'label': 'SSB'}])
handles.append(mlines.Line2D([], [], color='r', label='SSB Mean', linestyle='--'))
ax[0][1].legend(handles = handles)
ax[0][1].grid()


# Plot SSB and mortality recruitment
ax_10 =ax[1][0].twinx()
sns.scatterplot(data=fm_rs_ssb,
                x = "year",
                y="ssb", 
                color='g',
                ax = ax[1][0])
sns.lineplot(data=fm_rs_ssb, 
             x='year', 
             y='ssb', 
             color='g',
             ax = ax[1][0])
sns.scatterplot(data=fm_rs_ssb,
                x = "year",
                y="fm",
                color='r',
                ax = ax_10)
sns.lineplot(data=fm_rs_ssb, 
             x='year', 
             y='fm', 
             color='r', 
             ax = ax_10)
ax_10.set_ylabel("Fishing Mortality")
ax[1][0].set(xticks=year_series)
ax[1][0].set_xticklabels(ax[1][0].get_xticks(), rotation = 90)
ax[1][0].set_title("C) SSB and Fishing Mortality")
ax[1][0].set_xlabel("Years")
ax[1][0].set_ylabel("SSB")
handles = build_twin_lineplot_legend ([{'color': 'g', 'label': 'SSB'}, 
                             {'color': 'r', 'label': 'Fishing Mortality'}])
ax[1][0].legend(handles = handles)
ax[1][0].grid()
plt.subplots_adjust(hspace=0.4, wspace=0.35)



In [ ]:
pw_fit.summary()

In [ ]:
fm_rs_ssb.sort_values(by="ssb")

In [ ]:
fm_rs_ssb.ssb.mean()

In [ ]:
[x for x in range (8, 1, -1)]

In [ ]:
round(4.757, 2)

In [ ]:
import numpy as np
matrix = [[-2,  5,  3,  2],
          [ 9, -6,  5,  1],
          [ 3,  2,  7,  3],
          [-1,  8, -4,  8]]
matrix = np.array(matrix)
matrix

In [ ]:
1- fm_rs_ssb.ssb